In [42]:

!pip install -qqq tiktoken
!pip install -qqq pinecone-client
!pip install -qqq pypdf
!pip install -qqq openai


In [3]:
!pip install -qU langchain-openai

In [4]:

from google.colab import userdata
OPENAI_API_KEY=userdata.get('openai-key')
PINECONE_API_KEY=userdata.get('pinecone-key')
!pip install -qqq langchain_community
from langchain_community.document_loaders.pdf import PyPDFDirectoryLoader


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 5.6 MB/s eta 0:00:00


In [5]:
import os
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY

In [10]:
!pip install PyPDF2
from PyPDF2 import PdfReader
import os

def read_doc(directory: str) -> list[str]:
    # Function to read the PDFs from a directory.

    # Args:
    # directory (str): The path of the directory where the PDFs are stored.

    # Returns:
    # list[str]: A list of text in the PDFs.

    page_contents = []

    # List all PDF files in the directory
    for filename in os.listdir(directory):
        if filename.endswith('.pdf'):
            filepath = os.path.join(directory, filename)

            # Open the PDF file
            reader = PdfReader(filepath)
            num_pages = len(reader.pages)

            # Extract text from each page
            for page_num in range(num_pages):
                page = reader.pages[page_num]
                page_contents.append(page.extract_text())

    return page_contents

# Call the function
full_document = read_doc("/content")
print(len(full_document))  # This should print the number of pages read


143


##Code for Creating the Embeddings for the whole file
###But my ram is getting crashed so I used only 15 pages in my experiment

In [ ]:
!pip install langchain_openai
from langchain_openai import OpenAIEmbeddings
embed_model = OpenAIEmbeddings(model="text-embedding-3-small")

def generate_embeddings(documents: list[any]) -> list[list[float]]:

    #Generate embeddings for a list of documents.

    #Args:
        #documents (list[any]): A list of document objects, each containing a 'page_content' attribute.

    #Returns:
        #list[list[float]]: A list containig a list of embeddings corresponding to the documents.

    embedded = [embed_model.embed_documents(doc) for doc in documents]
    return embedded


# Run the function
chunked_document_embeddings = generate_embeddings(documents=full_document)

# Let's see the dimension of our embedding model so we can set it up later in pinecone
print(len(chunked_document_embeddings))

##Embeddings for only 15 pages





In [11]:
!pip install langchain_openai
from langchain_openai import OpenAIEmbeddings

# Initialize the embedding model
embed_model = OpenAIEmbeddings(model="text-embedding-3-small")

def generate_embeddings(documents: list[str]) -> list[list[float]]:
    # Generate embeddings for a list of documents.
    # Args:
    # documents (list[str]): A list of document objects, each containing a 'page_content' attribute.
    # Returns:
    # list[list[float]]: A list containing a list of embeddings corresponding to the documents.

    embedded = [embed_model.embed_documents([doc]) for doc in documents]
    return embedded

# Extract the first 15 pages from full_document
selected_documents = full_document[:15]

# Run the function
chunked_document_embeddings = generate_embeddings(documents=selected_documents)

# Let's see the dimension of our embedding model so we can set it up later in pinecone
print(len(chunked_document_embeddings))

# Optionally, print the shape of one of the embeddings
print(len(chunked_document_embeddings[0]))


15
1


In [12]:
# create unique IDs
ids = [str(x) for x in range(0,len(chunked_document_embeddings))]

def combine_vector_and_text(
    documents: list[any], doc_embeddings: list[list[float]]) -> list[dict[str, any]]:
    """
    Process a list of documents along with their embeddings.

    Args:
    - documents (List[Any]): A list of documents (strings or other types).
    - doc_embeddings (List[List[float]]): A list of embeddings corresponding to the documents.

    Returns:
    - data_with_metadata (List[Dict[str, Any]]): A list of dictionaries, each containing an ID, embedding values, and metadata.
    """
    data_with_metadata = []

    for id,doc_text, embedding in zip(ids,documents, doc_embeddings):
        # Convert doc_text to string if it's not already a string
        if not isinstance(doc_text, str):
            doc_text = str(doc_text)

        # Generate a unique ID based on the text content
        doc_id = id

        # Create a data item dictionary
        data_item = {
            "id": doc_id,
            "values": embedding[0],
            "metadata": {"text": doc_text},  # Include the text as metadata
        }

        # Append the data item to the list
        data_with_metadata.append(data_item)

    return data_with_metadata


# Call the function
all_meta_data = combine_vector_and_text(full_document, chunked_document_embeddings)

In [13]:
from pinecone import Pinecone,ServerlessSpec
pinecone = Pinecone()

INDEX_NAME="rag-pdf"
if INDEX_NAME in [index.name for index in pinecone.list_indexes()]:
  pinecone.delete_index(INDEX_NAME)

pinecone.create_index(name=INDEX_NAME, dimension=1536, metric='cosine',
  spec=ServerlessSpec(cloud='aws', region='us-east-1'))

index = pinecone.Index(INDEX_NAME)

In [14]:
print(index)

In [15]:
index.upsert(all_meta_data)



{'upserted_count': 15}

In [16]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
# Generate embeddings for the query
def query_pinecone_index(query_embeddings: list, top_k=3, include_metadata: bool = True) -> dict:
    """
    Query a Pinecone index.

    Args:
    - query_embeddings (list): List of query embeddings.
    - top_k (int): Number of nearest neighbors to retrieve (default: 3).
    - include_metadata (bool): Whether to include metadata in the query response (default: True).

    Returns:
    - query_response (dict): Query response containing nearest neighbors.
    """
    query_response = index.query(
        vector=query_embeddings, top_k=top_k, include_metadata=include_metadata
    )
    return query_response

In [79]:
def handle_query_with_red_teaming(query: str, threshold: float = 0.3) -> str:
    """
    Handle the query by determining if it is on-topic or off-topic using red teaming.

    Args:
    - query (str): User's query.
    - threshold (float): Similarity threshold to classify questions as on-topic or off-topic.

    Returns:
    - str: Response to the query.
    """
    # Generate embedding for the query
    query_embedding = embed_model.embed_query(query)

    # Query the Pinecone index
    query_response = query_pinecone_index(query_embeddings=query_embedding)

    # Extract the similarities and metadata
    similarities = [result['score'] for result in query_response['matches']]
    documents = [result['metadata']['text'] for result in query_response['matches']]

    # Check if the query is on-topic
    max_similarity = max(similarities)
    if max_similarity < threshold:
        return "The question is beyond the scope of the current documents."

    # Find the most relevant document chunk
    most_relevant_index = np.argmax(similarities)
    return f"{documents[most_relevant_index]}"


In [75]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

LLM = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

# Extract only the text from the dictionary before passing it to the LLM

prompt = f"{response} Using the provided information, give me a summarized answer"

def query_response(prompt: str) -> str:
    """This function returns a better response using LLM
    Args:
        prompt (str): The prompt template

    Returns:
        str: The actual response returned by the LLM
    """
    answer = LLM.invoke(prompt)
    return answer.content



##Query1

In [83]:
# Example usage
query = "What is the score of the match between India and Pakistan?"
response = handle_query_with_red_teaming(query)
if response == "The question is beyond the scope of the current documents.":
  print("The question is beyond the scope of the current documents.")
else:
  from langchain_openai import ChatOpenAI
  from langchain.prompts import PromptTemplate

  LLM = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

  # Extract only the text from the dictionary before passing it to the LLM

  prompt = f"{response} Using the provided information, give me a summarized answer"

  def query_response(prompt: str) -> str:
      """This function returns a better response using LLM
      Args:
          prompt (str): The prompt template

      Returns:
          str: The actual response returned by the LLM
      """
      answer = LLM.invoke(prompt)
      return answer.content

  # Printing response from red teaming technique
  print(f"\n Response from the RAG: \n {response}")



  # Call the LLM function
  final_answer = query_response(prompt=prompt)
  print(f"\n Response from the LLM: \n {final_answer}")



 Response from the RAG: 
 The question is beyond the scope of the current documents.

 Response from the LLM: 
 I'm sorry, but without specific information or context, I am unable to provide a summarized answer to a question that is beyond the scope of the current documents.


##Query2

In [87]:
# Example usage
query = "What were Lufthansa's net profits and operating profits in 2004, and how do these figures compare to 2003?"
response = handle_query_with_red_teaming(query)
if response == "The question is beyond the scope of the current documents.":
  print("The question is beyond the scope of the current documents.")
else:
  from langchain_openai import ChatOpenAI
  from langchain.prompts import PromptTemplate

  LLM = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

  # Extract only the text from the dictionary before passing it to the LLM

  prompt = f"{response} Using the provided information, give me a summarized answer"

  def query_response(prompt: str) -> str:
      """This function returns a better response using LLM
      Args:
          prompt (str): The prompt template

      Returns:
          str: The actual response returned by the LLM
      """
      answer = LLM.invoke(prompt)
      return answer.content

  # Printing response from red teaming technique
  print(f"\n Response from the RAG: \n {response}")



  # Call the LLM function
  final_answer = query_response(prompt=prompt)
  print(f"\n Response from the LLM: \n {final_answer}")



 Response from the RAG: 
 Lufthansa flies back to profit  
 
German airline Lufthansa has returned to profit in 2004 after posting huge losses in 2003.  
In a preliminary report, the airline announced net profits of 400m euros 
($527.61m; Â£274.73m), compared with a loss of 984m euros in 2003. 
Operating profits were at 380m euros, ten times more than in 2003. 
Lufthansa was hit in 2003 by tough competition and a dip in demand following the Iraq war and the killer SARS virus. It was also hit by troubles at its US catering business. Last year, Lufthansa showed signs of recovery even as some European and US airlines were teetering on the brink of bankruptcy. The board of Lufthansa has recommended paying a 2004 
dividend of 0.30 euros per share. In 2003, shareholders did not get a 
dividend. The company said that it will give all the details of its 2004 
results on 23 March.  
 
-------------------------------------------------------------------------
------- 
---------------------------

##Query3

In [85]:
# Example usage
query = "What the result of yesterday t20 crciket worldcup seminal match?"
response = handle_query_with_red_teaming(query)
if response == "The question is beyond the scope of the current documents.":
  print("The question is beyond the scope of the current documents.")
else:
  from langchain_openai import ChatOpenAI
  from langchain.prompts import PromptTemplate

  LLM = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

  # Extract only the text from the dictionary before passing it to the LLM

  prompt = f"{response} Using the provided information, give me a summarized answer"

  def query_response(prompt: str) -> str:
      """This function returns a better response using LLM
      Args:
          prompt (str): The prompt template

      Returns:
          str: The actual response returned by the LLM
      """
      answer = LLM.invoke(prompt)
      return answer.content

  # Printing response from red teaming technique
  print(f"\n Response from the RAG: \n {response}")



  # Call the LLM function
  final_answer = query_response(prompt=prompt)
  print(f"\n Response from the LLM: \n {final_answer}")


The question is beyond the scope of the current documents.


##Query4

In [86]:
# Example usage
query = "How did Japan's industrial output and retail sales perform in January, and what impact did this have on economic recovery hopes and the stock market?"
response = handle_query_with_red_teaming(query)
if response == "The question is beyond the scope of the current documents.":
  print("The question is beyond the scope of the current documents.")
else:
  from langchain_openai import ChatOpenAI
  from langchain.prompts import PromptTemplate

  LLM = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

  # Extract only the text from the dictionary before passing it to the LLM

  prompt = f"{response} Using the provided information, give me a summarized answer"

  def query_response(prompt: str) -> str:
      """This function returns a better response using LLM
      Args:
          prompt (str): The prompt template

      Returns:
          str: The actual response returned by the LLM
      """
      answer = LLM.invoke(prompt)
      return answer.content

  # Printing response from red teaming technique
  print(f"\n Response from the RAG: \n {response}")



# Call the LLM function
final_answer = query_response(prompt=prompt)
print(f"\n Response from the LLM: \n {final_answer}")



 Response from the RAG: 
 Analysts said that the move is aimed at winning over investors opposed to 
its bid for the London Stock Exchange. Critics of the takeover have complained that the money could be better used by returning cash to shareholders. Deutsche Boerse also said profit in the three months to 31 December was 120.7m euros ($158.8m; Â£83.3m). Sales climbed to 364.4m 
euros, lifting revenue for the year to a record 1.45bn euros.  
 
Frankfurt -based Deutsche Boerse has offered Â£1.3bn ($2.48bn; 1.88bn 
euros) for the London Stock Exchange. Rival pan -European bourse Euronext 
is working also on a bid. Late on Monday, Deutsche Boerse said it would 
lift its 2004 dividend payment to 70 euro cents (Â£0.48; $0.98) from 55 euro cents a year earlier. "There is a whiff of a sweetener in there," Anais Faraj, an analyst at Nomura told the BBC's World Business Report. 
"Most of the disgruntled shareholders of Deutsche Boerse are complaining 
that the money that is being used for the b